In [23]:
import pandas as pd
df = pd.read_csv("Final.csv").dropna()

In [ ]:
corr = df.corr()
corr.style.background_gradient(cmap='coolwarm')

In [24]:
df.head()

,src,dst,sport,dport,proto,smallpacket,packetratio,initlen,Botresp,label
0,172.16.0.12,198.155.242.22,1035,80,6,3,0.000000,48,1.000000,1.0
1,172.16.0.12,64.95.58.150,1036,80,6,5778,0.618034,48,0.618034,1.0
2,64.95.58.150,172.16.0.12,80,1036,6,9349,0.589054,48,0.629305,0.0
3,172.16.0.12,89.3.63.62,1038,80,6,5778,0.618034,48,0.618034,1.0
4,89.3.63.62,172.16.0.12,80,1038,6,9349,0.572159,48,0.636068,0.0


In [25]:
df_packet = df.drop(['src','dst','sport','dport'],1)

In [26]:
df_packet.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 128378 entries, 0 to 128384
Data columns (total 6 columns):
proto          128378 non-null int64
smallpacket    128378 non-null int64
packetratio    128378 non-null float64
initlen        128378 non-null int64
Botresp        128378 non-null float64
label          128378 non-null float64
dtypes: float64(3), int64(3)
memory usage: 6.9 MB


In [27]:
df_packet.drop(['proto'],1).corr()

,smallpacket,packetratio,initlen,Botresp,label
smallpacket,1.000000,0.184828,-0.054337,-0.125901,-0.031557
packetratio,0.184828,1.000000,-0.217304,-0.923651,-0.010927
initlen,-0.054337,-0.217304,1.000000,0.246349,0.038093
Botresp,-0.125901,-0.923651,0.246349,1.000000,0.036058
label,-0.031557,-0.010927,0.038093,0.036058,1.000000


In [28]:
df_packet_unique =  df_packet.drop_duplicates()

In [29]:
print(df_packet_unique.shape)
print(df_packet.shape)

(5642, 6)
(128378, 6)


In [11]:
df_packet_unique.drop(['proto'],1).corr()

,smallpacket,packetratio,initlen,Botresp,label
smallpacket,1.000000,0.189069,0.008625,-0.121812,-0.015109
packetratio,0.189069,1.000000,-0.068043,-0.766092,-0.047810
initlen,0.008625,-0.068043,1.000000,0.110268,0.017557
Botresp,-0.121812,-0.766092,0.110268,1.000000,0.088283
label,-0.015109,-0.047810,0.017557,0.088283,1.000000


In [32]:
from sklearn import feature_selection
feature_selection.chi2(df_packet_unique.drop(['proto','label'],1),df_packet_unique['label'].astype('int'))

(array([2.67834521e+03, 5.39548473e+00, 3.48599882e+01, 1.41649187e+00]),
 array([0.00000000e+00, 2.01889170e-02, 3.54287350e-09, 2.33981688e-01]))

In [46]:
from sklearn.feature_selection import SelectKBest,f_classif
X,y = (df_packet_unique.drop(['proto','label'],1),df_packet_unique['label'].astype('int'))
SelectKBest(f_classif,k='all').fit(X,y).score_func(X,y)

(array([ 1.28779086, 12.92127609,  1.73902832, 44.30262696]),
 array([2.56503692e-01, 3.27643670e-04, 1.87315536e-01, 3.07963174e-11]))

In [47]:
from sklearn.feature_selection import SelectKBest,mutual_info_classif
X,y = (df_packet_unique.drop(['proto','label'],1),df_packet_unique['label'].astype('int'))
SelectKBest(mutual_info_classif,k='all').fit(X,y).score_func(X,y)

array([0.09190278, 0.13622245, 0.06329966, 0.15190608])

In [49]:
df_corr = df_packet_unique.drop(['proto','smallpacket','initlen'],1)

In [50]:
df_corr.head()

,packetratio,Botresp,label
0,0.000000,1.000000,1.0
1,0.618034,0.618034,1.0
2,0.589054,0.629305,0.0
3,0.618034,0.618034,1.0
4,0.572159,0.636068,0.0


In [51]:
df_corr['label'].value_counts()

0.0    3911
1.0    1731
Name: label, dtype: int64

In [56]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(df_corr.drop(['label'],1),df_corr['label'],test_size=0.3,random_state=123)

In [57]:
print(y_train.value_counts())
print(y_test.value_counts())

0.0    2711
1.0    1238
Name: label, dtype: int64
0.0    1200
1.0     493
Name: label, dtype: int64


In [58]:
from sklearn.linear_model import LogisticRegression

In [62]:
lr = LogisticRegression(random_state=123,class_weight='balanced',verbose=True)

In [63]:
lr.fit(X_train,y_train)

[LibLinear]

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='warn', n_jobs=None, penalty='l2', random_state=123,
          solver='warn', tol=0.0001, verbose=True, warm_start=False)

In [64]:
pred = lr.predict(X_test)

In [65]:
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_test,pred))
print(classification_report(y_test,pred))

[[703 497]
 [271 222]]
              precision    recall  f1-score   support

         0.0       0.72      0.59      0.65      1200
         1.0       0.31      0.45      0.37       493

   micro avg       0.55      0.55      0.55      1693
   macro avg       0.52      0.52      0.51      1693
weighted avg       0.60      0.55      0.57      1693



In [78]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(hidden_layer_sizes=(50,40),verbose=True,random_state=123)
mlp.fit(X_train,y_train)

Iteration 1, loss = 0.68490730
Iteration 2, loss = 0.62999301
Iteration 3, loss = 0.62212077
Iteration 4, loss = 0.62035246
Iteration 5, loss = 0.61874387
Iteration 6, loss = 0.61791816
Iteration 7, loss = 0.61646399
Iteration 8, loss = 0.61581522
Iteration 9, loss = 0.61519564
Iteration 10, loss = 0.61452682
Iteration 11, loss = 0.61455133
Iteration 12, loss = 0.61437625
Iteration 13, loss = 0.61361685
Iteration 14, loss = 0.61353240
Iteration 15, loss = 0.61354236
Iteration 16, loss = 0.61312275
Iteration 17, loss = 0.61289260
Iteration 18, loss = 0.61253669
Iteration 19, loss = 0.61238976
Iteration 20, loss = 0.61208707
Iteration 21, loss = 0.61188516
Iteration 22, loss = 0.61176125
Iteration 23, loss = 0.61155146
Iteration 24, loss = 0.61139072
Iteration 25, loss = 0.61123780
Iteration 26, loss = 0.61054725
Iteration 27, loss = 0.61024470
Iteration 28, loss = 0.61048971
Iteration 29, loss = 0.61000881
Iteration 30, loss = 0.60938414
Iteration 31, loss = 0.60974181
Iteration 32, los

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(50, 40), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=123, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=True, warm_start=False)

In [79]:
pred = mlp.predict(X_test)

In [80]:
from sklearn.metrics import classification_report,confusion_matrix


[[1118   82]
 [ 378  115]]
              precision    recall  f1-score   support

         0.0       0.75      0.93      0.83      1200
         1.0       0.58      0.23      0.33       493

   micro avg       0.73      0.73      0.73      1693
   macro avg       0.67      0.58      0.58      1693
weighted avg       0.70      0.73      0.68      1693



In [81]:
def report(y_test,pred):
    print(confusion_matrix(y_test,pred))
    print(classification_report(y_test,pred))

In [82]:
X_packet_train,X_packet_test,y_packet_train,y_packet_test = train_test_split(df_packet.drop(['label','proto'],1),df_packet['label'],test_size=0.3,random_state=123)

In [83]:
mlp = MLPClassifier(hidden_layer_sizes=(50,40),verbose=True,random_state=123)
mlp.fit(X_packet_train,y_packet_train)

Iteration 1, loss = 0.65678903
Iteration 2, loss = 0.58384133
Iteration 3, loss = 0.51859259
Iteration 4, loss = 0.48908604
Iteration 5, loss = 0.46307944
Iteration 6, loss = 0.45286477
Iteration 7, loss = 0.43134487
Iteration 8, loss = 0.40943266
Iteration 9, loss = 0.40142575
Iteration 10, loss = 0.39335394
Iteration 11, loss = 0.38076854
Iteration 12, loss = 0.37711979
Iteration 13, loss = 0.38273457
Iteration 14, loss = 0.35882716
Iteration 15, loss = 0.37102526
Iteration 16, loss = 0.35421079
Iteration 17, loss = 0.35116719
Iteration 18, loss = 0.34499374
Iteration 19, loss = 0.34986405
Iteration 20, loss = 0.33313701
Iteration 21, loss = 0.32932145
Iteration 22, loss = 0.33399470
Iteration 23, loss = 0.31699130
Iteration 24, loss = 0.31708506
Iteration 25, loss = 0.31762126
Iteration 26, loss = 0.30741351
Iteration 27, loss = 0.30081305
Iteration 28, loss = 0.30173599
Iteration 29, loss = 0.29191143
Iteration 30, loss = 0.28662026
Iteration 31, loss = 0.28042619
Iteration 32, los

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(50, 40), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=123, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=True, warm_start=False)

In [85]:
pred = mlp.predict(X_packet_test)
report(y_packet_test,pred)

[[20328   892]
 [ 1341 15953]]
              precision    recall  f1-score   support

         0.0       0.94      0.96      0.95     21220
         1.0       0.95      0.92      0.93     17294

   micro avg       0.94      0.94      0.94     38514
   macro avg       0.94      0.94      0.94     38514
weighted avg       0.94      0.94      0.94     38514



In [87]:
print(y_packet_train.value_counts())
print(y_packet_test.value_counts())

0.0    49850
1.0    40014
Name: label, dtype: int64
0.0    21220
1.0    17294
Name: label, dtype: int64


In [88]:
df_packet_unique.head()

,proto,smallpacket,packetratio,initlen,Botresp,label
0,6,3,0.000000,48,1.000000,1.0
1,6,5778,0.618034,48,0.618034,1.0
2,6,9349,0.589054,48,0.629305,0.0
3,6,5778,0.618034,48,0.618034,1.0
4,6,9349,0.572159,48,0.636068,0.0


In [89]:
Xu_packet_train,Xu_packet_test,yu_packet_train,yu_packet_test = train_test_split(df_packet_unique.drop(['label','proto'],1),df_packet_unique['label'],test_size=0.3,random_state=123)

In [92]:
mlpu = MLPClassifier(hidden_layer_sizes=(20,30),verbose=True,random_state=123)
mlpu.fit(Xu_packet_train,yu_packet_train)

Iteration 1, loss = 0.97297904
Iteration 2, loss = 0.73601116
Iteration 3, loss = 0.71274432
Iteration 4, loss = 0.65376830
Iteration 5, loss = 0.76925862
Iteration 6, loss = 0.73381023
Iteration 7, loss = 0.66662109
Iteration 8, loss = 0.64033869
Iteration 9, loss = 0.62999265
Iteration 10, loss = 0.65958385
Iteration 11, loss = 0.72488104
Iteration 12, loss = 0.65632810
Iteration 13, loss = 0.67692553
Iteration 14, loss = 0.61501982
Iteration 15, loss = 0.63337182
Iteration 16, loss = 0.61518514
Iteration 17, loss = 0.63551191
Iteration 18, loss = 0.76752533
Iteration 19, loss = 0.67684881
Iteration 20, loss = 0.66052872
Iteration 21, loss = 0.74140625
Iteration 22, loss = 0.66549252
Iteration 23, loss = 0.63387439
Iteration 24, loss = 0.62664850
Iteration 25, loss = 0.60911211
Iteration 26, loss = 0.65624747
Iteration 27, loss = 0.62034398
Iteration 28, loss = 0.61083656
Iteration 29, loss = 0.67366734
Iteration 30, loss = 0.63495785
Iteration 31, loss = 0.65185674
Iteration 32, los

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(50, 30), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=123, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=True, warm_start=False)

In [93]:
pred = mlpu.predict(Xu_packet_test)
report(yu_packet_test,pred)

[[1192    8]
 [ 481   12]]
              precision    recall  f1-score   support

         0.0       0.71      0.99      0.83      1200
         1.0       0.60      0.02      0.05       493

   micro avg       0.71      0.71      0.71      1693
   macro avg       0.66      0.51      0.44      1693
weighted avg       0.68      0.71      0.60      1693



In [94]:
print(yu_packet_train.value_counts())
print(yu_packet_test.value_counts())

0.0    2711
1.0    1238
Name: label, dtype: int64
0.0    1200
1.0     493
Name: label, dtype: int64


In [ ]:
#                           precision  recall     f1       
# packetratio, botresp       0.71      0.73      0.68
# all 4                      0.68      0.71      0.60 
# With duplication all 4     0.94      0.94      0.94